In [1]:
%matplotlib inline
import geopandas
import pandas
import os
import glob

import dateutil
import datetime

from simpledbf import Dbf5
import pandas_access as mdb


pandas.set_option("display.max_columns",999)

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

## STL Open Data Download Links

* Source: http://data.stlouis-mo.gov/downloads.cfm
* Not all links from source are included--more may in the future

In [4]:
download_links =[{'file':'codes','file_type':'access','daily_update':False,'url':'http://data.stlouis-mo.gov/downloads/codes.zip'},
                 {'file':'parcels','file_type':'shapefile','daily_update':False,'url':'http://data.stlouis-mo.gov/downloads/prcl_shape.zip'},
                 {'file':'blocks','file_type':'shapefile','daily_update':False,'url':'http://data.stlouis-mo.gov/downloads/blocks_shape.zip'},
                 {'file':'neighborhoods_wards','file_type':'shapefile','daily_update':False,'url':'http://data.stlouis-mo.gov/downloads/nbrhds_wards.zip'},
                 {'file':'historic_districs','file_type':'shapfile','daily_update':False,'url':'http://data.stlouis-mo.gov/downloads/historic_dist.zip'},
                 {'file':'streets','file_type':'shapefile','daily_update':False,'url':'http://data.stlouis-mo.gov/downloads/streets.zip'},
                 {'file':'parcels_tax_records','file_type':'access','daily_update':True,'url':'http://data.stlouis-mo.gov/downloads/prcl.zip'},
                 {'file':'parcels_land_records','file_type':'dbf','daily_update':True,'url':'http://data.stlouis-mo.gov/downloads/par.zip'},
                 {'file':'parcel_sales','file_type':'access','daily_update':True,'url':'http://data.stlouis-mo.gov/downloads/prclsale.zip'},
                 {'file':'inspections','file_type':'access','daily_update':True,'url':'http://data.stlouis-mo.gov/downloads/bldginsp.zip'},
                 {'file':'service_requests','file_type':'access','daily_update':True,'url':'http://data.stlouis-mo.gov/downloads/csb.zip'},
                ]
                 
                 
download_links = pandas.DataFrame(download_links)
download_links.loc[:,'dest_zip_location'] = download_links.url.apply(lambda x: os.path.join('data','open-data-downloads',x.split('/')[-1]))  
download_links.loc[:,'dest_unzip_location'] = download_links.dest_zip_location.apply(lambda x: x.split('.zip')[0].replace('downloads','extracted'))
download_links

,daily_update,file,file_type,url,dest_zip_location,dest_unzip_location
0,False,codes,access,http://data.stlouis-mo.gov/downloads/codes.zip,data/open-data-downloads/codes.zip,data/open-data-extracted/codes
1,False,parcels,shapefile,http://data.stlouis-mo.gov/downloads/prcl_shap...,data/open-data-downloads/prcl_shape.zip,data/open-data-extracted/prcl_shape
2,False,blocks,shapefile,http://data.stlouis-mo.gov/downloads/blocks_sh...,data/open-data-downloads/blocks_shape.zip,data/open-data-extracted/blocks_shape
3,False,neighborhoods_wards,shapefile,http://data.stlouis-mo.gov/downloads/nbrhds_wa...,data/open-data-downloads/nbrhds_wards.zip,data/open-data-extracted/nbrhds_wards
4,False,historic_districs,shapfile,http://data.stlouis-mo.gov/downloads/historic_...,data/open-data-downloads/historic_dist.zip,data/open-data-extracted/historic_dist
5,False,streets,shapefile,http://data.stlouis-mo.gov/downloads/streets.zip,data/open-data-downloads/streets.zip,data/open-data-extracted/streets
6,True,parcels_tax_records,access,http://data.stlouis-mo.gov/downloads/prcl.zip,data/open-data-downloads/prcl.zip,data/open-data-extracted/prcl
7,True,parcels_land_records,dbf,http://data.stlouis-mo.gov/downloads/par.zip,data/open-data-downloads/par.zip,data/open-data-extracted/par
8,True,parcel_sales,access,http://data.stlouis-mo.gov/downloads/prclsale.zip,data/open-data-downloads/prclsale.zip,data/open-data-extracted/prclsale
9,True,inspections,access,http://data.stlouis-mo.gov/downloads/bldginsp.zip,data/open-data-downloads/bldginsp.zip,data/open-data-extracted/bldginsp


## Download and Extract Data

In [24]:
import urllib
import zipfile



today_only = True

if today_only:
    download_subset = [True]
else: download_subset = [True,False]

# Download files
download_links[download_links.daily_update.isin(download_subset)].apply(lambda x: urllib.request.urlretrieve(x.url,x.dest_zip_location),axis=1)

# Extract files
download_links[download_links.daily_update.isin(download_subset)].apply(lambda x: zipfile.ZipFile(x.dest_zip_location).extractall(x.dest_unzip_location),axis=1)

0     (data/open-data-downloads/codes.zip, [Content-...
1     (data/open-data-downloads/prcl_shape.zip, [Con...
2     (data/open-data-downloads/blocks_shape.zip, [C...
3     (data/open-data-downloads/nbrhds_wards.zip, [C...
4     (data/open-data-downloads/historic_dist.zip, [...
5     (data/open-data-downloads/streets.zip, [Conten...
6     (data/open-data-downloads/prcl.zip, [Content-T...
7     (data/open-data-downloads/par.zip, [Content-Ty...
8     (data/open-data-downloads/prclsale.zip, [Conte...
9     (data/open-data-downloads/bldginsp.zip, [Conte...
10    (data/open-data-downloads/csb.zip, [Content-Ty...
dtype: object

## List usable files

In [7]:
print("## Access Databases and Tables")
access_dbs = glob.glob('data/open-data-extracted/*/*.mdb')
for i in access_dbs:
    print(i)
    print(mdb.list_tables(i))
    print()
print("\n## Shapefiles")
shps = glob.glob('data/open-data-extracted/*/*.shp')
for i in shps:
    print(i)
    

## Access Databases and Tables
data/open-data-extracted/prclsale/prclsale.mdb
['CdCityBlockPartSrc', 'CdSaleType', 'HistPrclSale', 'PrclSale']

data/open-data-extracted/bldginsp/bldginsp.mdb
['BldgInsp', 'BldgInspDate', 'Condemn', 'CondemnDate', 'LtrDate', 'LtrOwner', 'VacBldg', 'VacBldgDate']

data/open-data-extracted/prcl/prcl.mdb
['BldgCom', 'BldgRes', 'BldgResImp', 'BldgSect', 'CxPrclCnBlk10', 'Prcl', 'PrclAddr', 'PrclAddrLRMS', 'PrclAsmt', 'PrclAttr', 'PrclDate', 'PrclImp', 'PrclREAR']

data/open-data-extracted/csb/cityworks.mdb
['dbo_vw_public']


## Shapefiles
data/open-data-extracted/blocks_shape/blocks.shp
data/open-data-extracted/nbrhds_wards/BND_Nhd88_cw.shp
data/open-data-extracted/nbrhds_wards/POL_WRD_2010_Prec.shp
data/open-data-extracted/prcl_shape/prcl.shp
data/open-data-extracted/streets/tgr_str_cl.shp
data/open-data-extracted/streets/TgrGeoCd.shp
